In [1]:
from pathlib import Path
import os

yr = 2023
d = 21

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = {}
  for i, l in enumerate(inp.splitlines()):
    for j, c in enumerate(l):
      formatted_input[(i,j)] = c
  return formatted_input

In [3]:
def find_walkable_plots(formatted_input, steps, progress_bar=False):
  import numpy as np
  from tqdm.notebook import tqdm


  neighbors = [np.array(x) for x in [[0,1], [0,-1], [1,0], [-1,0]]]
  current_locs = {k: True for k, v in formatted_input.items() if v=='S'}
  next_locs = {}
  for step in tqdm(list(range(steps))) if progress_bar else range(steps):
    next_locs = {}
    rejected_locs = {}
    for cl in current_locs:
      for o in neighbors:
        nl = tuple(np.add(np.array(cl), np.array(o)))
        if (nl not in next_locs
           and nl not in rejected_locs
           and formatted_input.get(nl, False) in ['.', 'S']):
          next_locs[nl] = True
        else:
          rejected_locs[nl] = True
    current_locs = next_locs
  return len(current_locs)

In [4]:
def find_walkable_plots_outer_diamond(formatted_input, n, progress_bar=False):
    
  def outer_diamond(formatted_input):
    import copy
    outer = copy.deepcopy(formatted_input)
    for i, ci in enumerate((list(range(65))+[65]+list(reversed(range(65))))):
      for ci2 in range(65-ci,65+ci+1):
        outer[(i,ci2)] = '#'
    return outer

  outer = outer_diamond(formatted_input)

  # We're assuming that we will be starting in the corners such that
  # we will be compatible with the inner diamonds
  outer[(0,1)] = 'S'
  outer[(0,129)] = 'S'
  outer[(129,0)] = 'S'
  outer[(129,130)] = 'S'

  return find_walkable_plots(outer, n, progress_bar=progress_bar)

In [5]:
def tiled_count(n):
  '''
  Starting at the center of the original input, with 64 steps we can reach 3617
  tiles. This leaves us with a diamond that extends to the row and columns BEFORE
  the original input ends (i.e. there is a 1 row/column gap). The input is set
  up so there is also a diamond that is formed by the corners of the input
  when they are put together. The middle diamond of 3617 can be adjacent to diamonds
  of count 3717.

  Then if we do one more step of the inner diamond, we get a count of 3703 and
  this diamond extends the entire way to the edge of the input. This version of
  the middle diamond can be adjacent to a corner diamond of count 3547.

  We know that this must be the configuration because otherwise we would end up
  with adjacent locations that are walkable, which is impossible.


  We then know that in each row the two diamond types must flip-flop in order
  to maintain a valid walkable set (no two adjacent squares are both walkable)

  It takes 65 steps to leave the first input, and depending on whether the total
  number of steps is even or odd, we will either end up with the 3617 diamond in
  the center or the 3703 diamond in the center.

  We can determine how many "tiles" of the input we have by subtracting 65 from
  the number of steps (to leave the original tile), then dividing by 131 to
  determine how many tiles to the left, right, up, and down will exist from the
  tile. i.e. (n-65)/131=2 means at the widest point the diamond has 5 tiles
  (2+2+1).

  The total number of tiles is then calculated as (((n-65)/131)+1)**2.

  We get the number of each type of inner diamond (3617 or 3703) by counting
  concentric diamonds out from the center. Each concentric diamond adds more
  tiles like [1, 4, 8, 12, 16, 20, 24, ..., 4*(m-1)] where m is the number of
  concentric diamonds. We alternate adding these depending on whether a
  3617 or 3703 was in the middle tile.

  Once we have determined how many "inner" diamonds we have, call this 'x'.
  The number of outer diamonds is the total number of tiles minus x.

  We will always have an equal number of outer diamond types, so to our running
  total we add (x/2)*3717 and (x/2)*3547.

  So the total is the number of each inner diamond times their count (2 of these),
  plus the number of outer diamonds times their counts (2 of these and the 2 counts
  will always be equal)
  '''
  import numpy as np

  assert((n-65)%131==0)
  n_tiles = int((n-65)/131)

  inner_diamond_even_count = find_walkable_plots(formatted_input, 64, progress_bar=False)
  assert(inner_diamond_even_count == 3617)
  inner_diamond_odd_count = find_walkable_plots(formatted_input, 65, progress_bar=False)
  assert(inner_diamond_odd_count == 3703)

  outer_diamond_even_count = find_walkable_plots_outer_diamond(formatted_input, 64, progress_bar=False)
  assert(outer_diamond_even_count == 3547)
  outer_diamond_odd_count = find_walkable_plots_outer_diamond(formatted_input, 65, progress_bar=False)
  assert(outer_diamond_odd_count == 3717)

  if n_tiles%2==0:
    order = (inner_diamond_odd_count, inner_diamond_even_count)
  else:
    order = (inner_diamond_even_count, inner_diamond_odd_count)


  s = np.array([0], np.ulonglong)
  t_counts = np.cumsum(np.array([4]*n_tiles, dtype=np.ulonglong)) # Number of tiles on each concentric diamond

  ti = 0
  s[0] += order[0]
  while(ti<n_tiles):

    s[0] += np.multiply(order[(ti+1)%2], t_counts[ti])
    ti+=1

  total_n_tiles = (2*n_tiles+1)**2
  n_opposite_tiles = total_n_tiles - np.sum(t_counts)-1
  assert(n_opposite_tiles%2==0)
  s[0] += (n_opposite_tiles/2) * 3547
  s[0] += (n_opposite_tiles/2) * 3717
  return s[0]

In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(find_walkable_plots(formatted_input, 64, progress_bar=False))
print(tiled_count(26501365))

print("\nRUNTIME: ", time.time()-t)

3617
596857397104703

RUNTIME:  3.9663913249969482
